# Mise en forme des données

Ici on prend les données original et on les mets dans currated

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Répertoire des données d'origine.

dataOriginalPath = 'Data/Original/'
dataTrainX = dataOriginalPath +'clickbait17-train-170630/instances.jsonl'
dataTrainY = dataOriginalPath +'clickbait17-train-170630/truth.jsonl'
dataTestX = dataOriginalPath +'clickbait17-test-170720/instances.jsonl'
dataTestY = dataOriginalPath +'clickbait17-test-170720/truth.jsonl'

# Le dataset va être sauvegardé là (on conserve tous les sites à cette étape du calcul) :
dataCuratedPath = 'Data/Curated/'
LCfraction = 'Fraction'

In [3]:
pdoTrainX=pd.read_json(dataTrainX, lines = True)

In [4]:
pdoTrainX

,postMedia,postText,id,targetCaptions,targetParagraphs,targetTitle,postTimestamp,targetKeywords,targetDescription
0,[],[UK’s response to modern slavery leaving victi...,858462320779026432,[modern-slavery-rex.jpg],[Thousands of modern slavery victims have not ...,‘Inexcusable’ failures in UK’s response to mod...,Sat Apr 29 23:25:41 +0000 2017,"modern slavery, Department For Work And Pensio...",“Inexcusable” failures in the UK’s system for ...
1,[],[this is good],858421020331560960,"[In this July 1, 2010 file photo, Dr. Charmain...",[President Donald Trump has appointed the pro-...,Donald Trump Appoints Pro-Life Advocate as Ass...,Sat Apr 29 20:41:34 +0000 2017,"Americans United for Life, Dr. Charmaine Yoest...",President Donald Trump has appointed pro-life ...
2,[],"[The ""forgotten"" Trump roast: Relive his bruta...",858368123753435136,[President Trump will not attend this year's W...,[When the White House correspondents’ dinner i...,The ‘forgotten’ Trump roast: Relive his brutal...,Sat Apr 29 17:11:23 +0000 2017,"trump whcd, whcd, white house correspondents d...",President Trump won't be at this year's White ...
3,[],[Meet the happiest #dog in the world!],858323428260139008,"[Maru , Maru, Maru, Maru, Maru]",[Adorable is probably an understatement. This ...,"Meet The Happiest Dog In The World, Maru The H...",Sat Apr 29 14:13:46 +0000 2017,"Maru, husky, dogs, pandas, furball, instagram","The article is about Maru, a husky dog who has..."
4,[],[Tokyo's subway is shut down amid fears over a...,858283602626347008,[All nine lines of Tokyo's subway system were ...,[One of Tokyo's major subways systems says it ...,Tokyo's subway is shut down amid fears over an...,Sat Apr 29 11:35:31 +0000 2017,"Tokyo,subway,shut,fears,North,Korean,attack","The temporary suspension, which lasted ten min..."
...,...,...,...,...,...,...,...,...,...
19533,[media/photo_804240867972304896.jpg],[Brazil soccer team and pilot's final intervie...,804250183642976256,"[CNBC, msnbc, NBC NEWS, TODAY, xfinity]",[Watch Live: Joe Biden Honored on Senate Floor...,"NBC News Video See Brazil Soccer Team, Pilot’s...",Thu Dec 01 09:06:00 +0000 2016,,NBC News
19534,[],[😱😱😱😱😱😱😱😱😱😱😱😱😱😱],804156272086020096,"[Instagram/madonna, Speaker Ryan Retreats on H...",[On November 30 Politico reported that Eric Tr...,Politico Scoop: Eric Trump Killed Two Deer,Thu Dec 01 02:52:50 +0000 2016,Politico Scoop: Eric Trump Killed Two Deer,Politico Scoop: Eric Trump Killed Two Deer
19535,[],[Frenchs Forest high school may have to make w...,804149798651588608,[An artist's impression of the proposed new to...,[The Forest High School on Sydney's northern b...,Frenchs Forest high school may relocate to mak...,Thu Dec 01 02:27:07 +0000 2016,"frenchs forest, northern beaches, sydney, rede...",The Forest High School on Sydney's northern be...
19536,[media/photo_804133521023324160.jpg],[Oh Jeff… #bruh],804134698729385984,[Jeff Fisher May Think Danny Woodhead Still Pl...,[NFL coaches have a lot of information to reme...,Los Angeles Rams Jeff Fisher May Think Danny W...,Thu Dec 01 01:27:06 +0000 2016,"Humor, Football, NFL, NFC West, Los Angeles Ra...","Los Angeles Rams news, rumors, scores, schedul..."


In [5]:
pdoTrainX.drop(['postMedia', 'targetCaptions', 'targetParagraphs', 'postTimestamp' , 'targetKeywords']	, axis=1, inplace=True)

In [6]:
pdoTrainX

,postText,id,targetTitle,targetDescription
0,[UK’s response to modern slavery leaving victi...,858462320779026432,‘Inexcusable’ failures in UK’s response to mod...,“Inexcusable” failures in the UK’s system for ...
1,[this is good],858421020331560960,Donald Trump Appoints Pro-Life Advocate as Ass...,President Donald Trump has appointed pro-life ...
2,"[The ""forgotten"" Trump roast: Relive his bruta...",858368123753435136,The ‘forgotten’ Trump roast: Relive his brutal...,President Trump won't be at this year's White ...
3,[Meet the happiest #dog in the world!],858323428260139008,"Meet The Happiest Dog In The World, Maru The H...","The article is about Maru, a husky dog who has..."
4,[Tokyo's subway is shut down amid fears over a...,858283602626347008,Tokyo's subway is shut down amid fears over an...,"The temporary suspension, which lasted ten min..."
...,...,...,...,...
19533,[Brazil soccer team and pilot's final intervie...,804250183642976256,"NBC News Video See Brazil Soccer Team, Pilot’s...",NBC News
19534,[😱😱😱😱😱😱😱😱😱😱😱😱😱😱],804156272086020096,Politico Scoop: Eric Trump Killed Two Deer,Politico Scoop: Eric Trump Killed Two Deer
19535,[Frenchs Forest high school may have to make w...,804149798651588608,Frenchs Forest high school may relocate to mak...,The Forest High School on Sydney's northern be...
19536,[Oh Jeff… #bruh],804134698729385984,Los Angeles Rams Jeff Fisher May Think Danny W...,"Los Angeles Rams news, rumors, scores, schedul..."


In [7]:
pdoTrainX.set_index('id')

,postText,targetTitle,targetDescription
id,,,
858462320779026432,[UK’s response to modern slavery leaving victi...,‘Inexcusable’ failures in UK’s response to mod...,“Inexcusable” failures in the UK’s system for ...
858421020331560960,[this is good],Donald Trump Appoints Pro-Life Advocate as Ass...,President Donald Trump has appointed pro-life ...
858368123753435136,"[The ""forgotten"" Trump roast: Relive his bruta...",The ‘forgotten’ Trump roast: Relive his brutal...,President Trump won't be at this year's White ...
858323428260139008,[Meet the happiest #dog in the world!],"Meet The Happiest Dog In The World, Maru The H...","The article is about Maru, a husky dog who has..."
858283602626347008,[Tokyo's subway is shut down amid fears over a...,Tokyo's subway is shut down amid fears over an...,"The temporary suspension, which lasted ten min..."
...,...,...,...
804250183642976256,[Brazil soccer team and pilot's final intervie...,"NBC News Video See Brazil Soccer Team, Pilot’s...",NBC News
804156272086020096,[😱😱😱😱😱😱😱😱😱😱😱😱😱😱],Politico Scoop: Eric Trump Killed Two Deer,Politico Scoop: Eric Trump Killed Two Deer
804149798651588608,[Frenchs Forest high school may have to make w...,Frenchs Forest high school may relocate to mak...,The Forest High School on Sydney's northern be...


In [31]:
pdoTrainX["postText"]=pdoTrainX["postText"].apply(lambda x : x[0])

In [32]:
pdoTrainY=pd.read_json(dataTrainY, lines = True)

In [33]:
pdoTrainY

,truthJudgments,truthMean,id,truthClass,truthMedian,truthMode
0,"[1.0, 1.0, 1.0, 1.0, 1.0]",1.000000,858464162594172928,clickbait,1.000000,1.000000
1,"[0.33333333330000003, 0.0, 0.33333333330000003...",0.133333,858462320779026432,no-clickbait,0.000000,0.000000
2,"[0.33333333330000003, 0.6666666666000001, 1.0,...",0.400000,858460992073863168,no-clickbait,0.333333,0.000000
3,"[0.0, 0.6666666666000001, 0.0, 0.3333333333000...",0.266667,858459539296980992,no-clickbait,0.333333,0.333333
4,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,858455355948384256,no-clickbait,0.000000,0.000000
...,...,...,...,...,...,...
19533,"[0.0, 0.6666666666000001, 0.0, 0.0, 0.0]",0.133333,804126501117435904,no-clickbait,0.000000,0.000000
19534,"[0.0, 0.0, 0.0, 0.33333333330000003, 0.0]",0.066667,804123103995580416,no-clickbait,0.000000,0.000000
19535,"[0.6666666666000001, 0.6666666666000001, 0.0, ...",0.333333,804121272967983104,no-clickbait,0.333333,0.000000
19536,"[1.0, 0.0, 0.6666666666000001, 1.0, 1.0]",0.733333,804119512010424320,clickbait,1.000000,1.000000


In [34]:
pdoTrainY.set_index('id')

,truthJudgments,truthMean,truthClass,truthMedian,truthMode
id,,,,,
858464162594172928,"[1.0, 1.0, 1.0, 1.0, 1.0]",1.000000,clickbait,1.000000,1.000000
858462320779026432,"[0.33333333330000003, 0.0, 0.33333333330000003...",0.133333,no-clickbait,0.000000,0.000000
858460992073863168,"[0.33333333330000003, 0.6666666666000001, 1.0,...",0.400000,no-clickbait,0.333333,0.000000
858459539296980992,"[0.0, 0.6666666666000001, 0.0, 0.3333333333000...",0.266667,no-clickbait,0.333333,0.333333
858455355948384256,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,no-clickbait,0.000000,0.000000
...,...,...,...,...,...
804126501117435904,"[0.0, 0.6666666666000001, 0.0, 0.0, 0.0]",0.133333,no-clickbait,0.000000,0.000000
804123103995580416,"[0.0, 0.0, 0.0, 0.33333333330000003, 0.0]",0.066667,no-clickbait,0.000000,0.000000
804121272967983104,"[0.6666666666000001, 0.6666666666000001, 0.0, ...",0.333333,no-clickbait,0.333333,0.000000


In [35]:
pdoTrainX['truthMean']=pdoTrainY['truthMean']

In [36]:
pdoTrainX

,postText,id,targetTitle,targetDescription,truthMean
0,UK’s response to modern slavery leaving victim...,858462320779026432,‘Inexcusable’ failures in UK’s response to mod...,“Inexcusable” failures in the UK’s system for ...,1.000000
1,this is good,858421020331560960,Donald Trump Appoints Pro-Life Advocate as Ass...,President Donald Trump has appointed pro-life ...,0.133333
2,"The ""forgotten"" Trump roast: Relive his brutal...",858368123753435136,The ‘forgotten’ Trump roast: Relive his brutal...,President Trump won't be at this year's White ...,0.400000
3,Meet the happiest #dog in the world!,858323428260139008,"Meet The Happiest Dog In The World, Maru The H...","The article is about Maru, a husky dog who has...",0.266667
4,Tokyo's subway is shut down amid fears over an...,858283602626347008,Tokyo's subway is shut down amid fears over an...,"The temporary suspension, which lasted ten min...",0.000000
...,...,...,...,...,...
19533,Brazil soccer team and pilot's final interview...,804250183642976256,"NBC News Video See Brazil Soccer Team, Pilot’s...",NBC News,0.133333
19534,😱😱😱😱😱😱😱😱😱😱😱😱😱😱,804156272086020096,Politico Scoop: Eric Trump Killed Two Deer,Politico Scoop: Eric Trump Killed Two Deer,0.066667
19535,Frenchs Forest high school may have to make wa...,804149798651588608,Frenchs Forest high school may relocate to mak...,The Forest High School on Sydney's northern be...,0.333333
19536,Oh Jeff… #bruh,804134698729385984,Los Angeles Rams Jeff Fisher May Think Danny W...,"Los Angeles Rams news, rumors, scores, schedul...",0.733333


In [37]:
# Il faut convertir le résultat sous forme de dataset
import datasets as ds
curated = ds.Dataset.from_pandas(pdoTrainX) #, preserve_index=False)

In [38]:
curated.train_test_split(0.2)

DatasetDict({
    train: Dataset({
        features: ['postText', 'id', 'targetTitle', 'targetDescription', 'truthMean'],
        num_rows: 15630
    })
    test: Dataset({
        features: ['postText', 'id', 'targetTitle', 'targetDescription', 'truthMean'],
        num_rows: 3908
    })
})

In [39]:
curated.info.description = "Clickbait"
curated.info.version = "0.0.1"
curated.info.supervised_keys = [LCfraction]
curated.save_to_disk(dataCuratedPath)